In [ ]:
# Input layer --> 3 Hidden Layers (128 -> 64 -> 32 neurons) --> Dropout (0.3) --> Output Layer
# Includes a custom MonotonicityLayer, STOCHASTIC MONOTONICITY, to enforce a soft constraint to the model
# Adam optimizer, learning rate = 0.001
# Loss function used is MSE, since it's a regression task

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Layer, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import logging

np.random.seed(42)
tf.random.set_seed(42)

logging.basicConfig(level=logging.INFO)

# Load the dataset
file_path = './alldata_downtownTodowntown.csv'
data = pd.read_csv(file_path)

# Check for NaN values
logging.info(f"NaN values in the dataset:\n{data.isna().sum()}")

# Handle NaN values
data = data.dropna()  # This removes rows with any NaN values

# Define our features and target
X = data.drop(columns=['total_number_trips', 'Unnamed: 0'])
y = data['total_number_trips']

# Check for NaN values again after preprocessing
logging.info(f"NaN values in X:\n{X.isna().sum()}")
logging.info(f"NaN values in y:\n{y.isna().sum()}")

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Normalize the target variable
target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Stochastic Monotonicity
class MonotonicityLayer(Layer):
    def __init__(self, lam, **kwargs):
        super(MonotonicityLayer, self).__init__(**kwargs)
        self.lam = lam

    def build(self, input_shape):
        self.w = self.add_weight(shape=(1,),
                                 initializer='ones',
                                 trainable=True,
                                 name='monotonicity_weight')
        super(MonotonicityLayer, self).build(input_shape)

    def call(self, inputs):
        features, predictions = inputs
        is_downtown = features[:, -1:]  # Ensure is_downtown is a 2D tensor

        # Soft monotonicity constraint
        downtown_effect = self.lam * tf.math.sigmoid(self.w * is_downtown)
        adjusted_predictions = predictions + downtown_effect

        return adjusted_predictions

# Create the model
def create_model(input_shape, lam):
    inputs = Input(shape=input_shape)
    x = Dense(128, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(1)(x)
    outputs = MonotonicityLayer(lam)([inputs, predictions])

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Train and evaluate the model
def train_and_evaluate(lam, batch_size):
    model = create_model(X_train.shape[1], lam)

    early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

    try:
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=100, batch_size=batch_size, callbacks=[early_stopping], verbose=0)

        predictions_scaled = model.predict(X_test)
        predictions = target_scaler.inverse_transform(predictions_scaled)

        # Check the monotonicity constraint
        is_downtown_test = X_test[:, -1] == 1
        violations = 0
        for i in range(len(predictions)):
            for j in range(i+1, len(predictions)):
                if is_downtown_test[i] > is_downtown_test[j] and predictions[i] < predictions[j]:
                    violations += 1

        mse = mean_squared_error(target_scaler.inverse_transform(y_test.reshape(-1, 1)), predictions)

        return mse, violations, history
    except Exception as e:
        logging.error(f"Error during model training or evaluation: {str(e)}")
        return None, None, None

# Test different values of lam
lam_values = [0, 0.2, 0.4, 0.6, 0.8, 1]
batch_size = 32
results = []

for lam in lam_values:
    mse, violations, history = train_and_evaluate(lam, batch_size)
    if mse is not None:
        results.append((lam, mse, violations))
        print(f"Lambda: {lam}, MSE: {mse}, Violations: {violations}")
    else:
        print(f"Skipping Lambda: {lam} due to error")

print("\nSummary of Results:")
for lam, mse, violations in results:
    print(f"Lambda: {lam}, MSE: {mse}, Violations: {violations}")

37/37 [==============================] - 0s 2ms/step
Lambda: 0, MSE: 7808.737672199453, Violations: 0
37/37 [==============================] - 0s 2ms/step
Lambda: 0.2, MSE: 7283.332736935329, Violations: 0
37/37 [==============================] - 0s 2ms/step
Lambda: 0.4, MSE: 6791.940853676783, Violations: 0
37/37 [==============================] - 0s 2ms/step
Lambda: 0.6, MSE: 9699.987798592872, Violations: 0
37/37 [==============================] - 0s 2ms/step
Lambda: 0.8, MSE: 7754.885316485795, Violations: 0
37/37 [==============================] - 0s 2ms/step
Lambda: 1, MSE: 7534.74428904993, Violations: 0

Summary of Results:
Lambda: 0, MSE: 7808.737672199453, Violations: 0
Lambda: 0.2, MSE: 7283.332736935329, Violations: 0
Lambda: 0.4, MSE: 6791.940853676783, Violations: 0
Lambda: 0.6, MSE: 9699.987798592872, Violations: 0
Lambda: 0.8, MSE: 7754.885316485795, Violations: 0
Lambda: 1, MSE: 7534.74428904993, Violations: 0
